In [16]:
import numpy as np

def MC_Asian(S0, dividends, vols, K, r, T, num_paths, seed = 100):
    
    np.random.seed(seed)
    N, M = len(S0), len(vols[0])
    dt = T / M
    Z = np.random.normal(size=(num_paths, N, M))
    S = np.zeros((num_paths, N, M + 1))
    S[:, :, 0] = S0

    for m in range(1, M + 1):
        vol_segment = vols[:, m - 1]
        drift = (r - dividends - 0.5 * vol_segment**2) * dt
        diffusion = vol_segment * np.sqrt(dt) * Z[:, :, m - 1]
        S[:, :, m] = S[:, :, m - 1] * np.exp(drift + diffusion)

    B_TM = np.mean(S[:, :, 1:] / S[:, :, 0, None], axis=1)
    basket_values = np.mean(B_TM, axis = 1)
    payoffs = np.maximum(basket_values - K, 0)
    price = np.exp(-r * T) * np.mean(payoffs)
    std = np.exp(-r * T) * np.std(payoffs) / np.sqrt(num_paths)
    
    return price, std

def FDM(S0, dividends, vols, K, r, T, num_paths, eps):
    N, M = len(S0), len(vols[0])
    S0_sensi = []
    price, _ = MC_Asian(S0, dividends, vols, K, r, T, num_paths)
    
    for i in range(N):
        S0[i] += eps
        new_price, _ = MC_Asian(S0, dividends, vols, K, r, T, num_paths)
        S0[i] -= eps
        S0_sensi.append((new_price - price) / eps)
    
    dividends_sensi = []
    for i in range(N):
        dividends[i] += eps
        new_price, _ = MC_Asian(S0, dividends, vols, K, r, T, num_paths)
        dividends[i] -= eps
        dividends_sensi.append((new_price - price) / eps)
    
    vols_sensi = []
    for i in range(N):
        sensi = []
        for j in range(M):
            vols[i][j] += eps
            new_price, _ = MC_Asian(S0, dividends, vols, K, r, T, num_paths)
            vols[i][j] -= eps
            sensi.append((new_price - price) / eps)
        vols_sensi.append(sensi)
    
    return price, S0_sensi, dividends_sensi, vols_sensi 
        

S0 = np.array([1.1, 1.2])
dividends = np.array([0.01, 0.02])
M = 10
vols = np.array([[0.1] * M, [0.2] * M])

T = 1
r = 0.05
K = 0.9
num_paths = 100

print(MC_Asian(S0, dividends, vols, K, r, T, num_paths))

(0.12034016035460982, 0.007035652963691115)


In [39]:
import numpy as np

def MC_Asian_AAD(S0, dividends, vols, K, r, T, num_paths, seed=100):
    """
    Compute the price and AAD-based sensitivities for an Asian basket option
    given by:
        Payoff = exp(-rT)*max((1/M)*sum_{m=1}^M B(T_m) - K, 0)
    where B(T_m) = (1/N)*sum_{i=1}^N exp(sum_{k=1}^m X_{i,k}) and
    X_{i,k} = (r - d_i - 0.5*vol_{i,k}^2)*dt + vol_{i,k}*sqrt(dt)*Z_{i,k}.

    Since the payoff is normalized by initial spots, dP/dS0 = 0.

    Returns:
    price: The estimated price
    dPdDiv: partial derivatives w.r.t. each asset's dividend (shape: N)
    dPdVol: partial derivatives w.r.t. each asset's vol steps (shape: N x M)
    """

    np.random.seed(seed)
    N, M = len(S0), vols.shape[1]
    dt = T / M

    # Generate random shocks
    Z = np.random.normal(size=(num_paths, N, M))

    # Preallocate arrays for X and exponentials
    # X[i,m] = increment at step m for asset i
    # We will store cumulative sums for convenience.
    X = np.zeros((num_paths, N, M))
    exp_sumX = np.zeros((num_paths, N, M))  # exp( sum_{k=1}^m X_{i,k} )

    # Forward pass: compute X_{i,m} and exp sums
    for m in range(M):
        vol_segment = vols[:, m]
        # X_{i,m} depends on d_i, sigma_{i,m}
        # X_{i,m} = (r - d_i - 0.5*vol^2)*dt + vol*sqrt(dt)*Z
        X[:, :, m] = (r - dividends - 0.5 * vol_segment**2)*dt + vol_segment*np.sqrt(dt)*Z[:, :, m]

    # Compute cumulative exponentials:
    # exp_sumX[i,m] = exp(X_{i,1} + X_{i,2} + ... + X_{i,m})
    # In code: we do a cumulative sum of X along m, then exponentiate.
    cumX = np.cumsum(X, axis=2)  # cumulative sum over time steps
    exp_sumX = np.exp(cumX)      # element-wise exp

    # Compute B(T_m) for each path and time m:
    # B(T_m) = (1/N)*sum_i exp_sumX[i,m]
    # shape of exp_sumX: (num_paths, N, M)
    B_Tm = np.mean(exp_sumX, axis=1)  # (num_paths, M)

    # Average over M:
    avg_B = np.mean(B_Tm, axis=1)  # (num_paths,)

    payoffs = np.maximum(avg_B - K, 0)
    price = np.exp(-r*T)*np.mean(payoffs)
    std = np.exp(-r*T)*np.std(payoffs)/np.sqrt(num_paths)

    # If the payoff is zero (OTM), derivative is zero. We must handle pathwise:
    # Derivatives only matter for paths with payoff>0.

    in_the_money = (avg_B > K)
    # dP/d(B(T_m)) = e^{-rT}*(1/M) if in the money, else 0
    dP_dB = np.zeros((num_paths, M))
    dP_dB[in_the_money, :] = np.exp(-r*T)*(1.0/M)

    # Now, dB(T_m)/dX_{i,k}:
    # B(T_m) = (1/N)*sum_i exp_sumX[i,m]
    # = (1/N)*sum_i exp(cumX[i,m])
    # dB(T_m)/dX_{i,k} = (1/N)*exp(cumX[i,m]) if k ≤ m else 0
    # We'll propagate this back.

    # dP/dX_{i,k} = sum_{m=k} dP/dB(T_m)*dB(T_m)/dX_{i,k}
    #             = sum_{m=k} [ e^{-rT}*(1/M) * (1/N)*exp_sumX[i,m] ]
    # for each path, asset i, and step k:
    dP_dX = np.zeros((num_paths, N, M))
    for m in range(M):
        # For each m, dB(T_m)/dX_{i,k} = (1/N)*exp_sumX[i,m] if k ≤ m, else 0
        # So we add contributions where k ≤ m.
        # rearranging sum: for k in [0,m], add e^{-rT}*(1/M)*(1/N)*exp_sumX[i,m]
        # We'll do a cumulative sum trick from the right:
        # Just loop k from 0 to m:
        factor = dP_dB[:, m][:, None]*(1.0/N)*exp_sumX[:, :, m]  # shape: (num_paths, N)
        # Add this factor to all k ≤ m:
        for k in range(m+1):
            dP_dX[:, :, k] += factor

    # Now apply chain rule for d_i and sigma_{i,k}:
    # X_{i,k} = ... => dX_{i,k}/dd_i = -dt
    # dX_{i,k}/d(sigma_{i,k}) = -sigma_{i,k}*dt + sqrt(dt)*Z_{i,k}

    dPdDiv = np.zeros((N,))  # derivative w.r.t. each dividend d_i
    dPdVol = np.zeros((N, M)) # derivative w.r.t. each vol sigma_{i,k}

    for i in range(N):
        for k in range(M):
            # sum over paths:
            # dP/dd_i = sum_paths dP/dX_{i,k} * dX_{i,k}/dd_i
            # dX_{i,k}/dd_i = -dt
            dPdDiv[i] += np.mean(dP_dX[:, i, k]*(-dt))
            # dP/dsigma_{i,k} = mean_paths of dP/dX_{i,k}*( -sigma_{i,k}*dt + sqrt(dt)*Z_{i,k} )
            dPdVol[i, k] = np.mean(dP_dX[:, i, k]*(-vols[i, k]*dt + np.sqrt(dt)*Z[:, i, k]))

    # dP/dS0[i] = 0 due to normalization (can return zeros)
    dPdS0 = np.zeros((N,))

    return price, std, dPdS0, dPdDiv, dPdVol


# Example usage matching the previous parameters:
S0 = np.array([1.1, 1.2])
dividends = np.array([0.01, 0.02])
M = 10
vols = np.array([[0.1]*M, [0.2]*M])
T = 1
r = 0.05
K = 0.9
num_paths = 100

price, std, dPdS0, dPdDiv, dPdVol = MC_Asian_AAD(S0, dividends, vols, K, r, T, num_paths)
print("Price:", price)
print("Std Error:", std)
print("dP/dS0:", dPdS0)       # Should be near zero
print("dP/dd:", dPdDiv)       # Sensitivity w.r.t. dividends
print("dP/dVol:", dPdVol)     # Sensitivity w.r.t. each vol step


Price: 0.12034016035460982
Std Error: 0.007035652963691118
dP/dS0: [0. 0.]
dP/dd: [-0.25780168 -0.25893111]
dP/dVol: [[-0.01777431  0.00162473  0.0134273   0.01370914  0.00886213 -0.00220983
   0.0087681   0.00572098  0.00051839 -0.00379171]
 [ 0.04099275  0.00872421  0.01036431 -0.0034463  -0.00127604 -0.00138699
  -0.00177295  0.00385868 -0.0036541  -0.00187849]]


In [40]:
price_fdm, S0_sensi_fdm, dividends_sensi_fdm, vols_sensi_fdm = FDM(S0, dividends, vols, K, r, T, num_paths, 1e-10)

In [44]:
vols_sensi_fdm

[[-0.017774115512736444,
  0.0016246726186608385,
  0.013427453593450878,
  0.013709172685949511,
  0.008862216516192234,
  -0.002209898930516374,
  0.00876826389273333,
  0.005720979245893432,
  0.0005181965967437918,
  -0.003791689184851066],
 [0.04099262596035658,
  0.008724410083260636,
  0.010364070712753914,
  -0.0034465486020707203,
  -0.0012763401446846956,
  -0.0013869461135129768,
  -0.001773026170326375,
  0.0038585801220847316,
  -0.0036541603076756246,
  -0.0018783585797876867]]

In [35]:
vols_sensi_fdm

[[-0.017774115512736444,
  0.0016246726186608385,
  0.013427453593450878,
  0.013709172685949511,
  0.008862216516192234,
  -0.002209898930516374,
  0.00876826389273333,
  0.005720979245893432,
  0.0005181965967437918,
  -0.003791689184851066],
 [0.04099262596035658,
  0.008724410083260636,
  0.010364070712753914,
  -0.0034465486020707203,
  -0.0012763401446846956,
  -0.0013869461135129768,
  -0.001773026170326375,
  0.0038585801220847316,
  -0.0036541603076756246,
  -0.0018783585797876867]]

In [36]:
vols_sensi_aad

array([[-1.31988561e-03,  7.79543194e-03,  6.53280127e-03,
         4.73371001e-03,  2.55559065e-03,  1.50460852e-03,
         1.63100827e-03,  1.15030824e-03, -1.31185854e-04,
         1.58206014e-04],
       [ 7.16998730e-02,  1.88040983e-01,  1.56449246e-03,
        -3.55045585e-03,  1.13526995e-02, -3.66470694e-03,
        -6.20071905e-04,  1.28822446e-03,  3.06494642e-03,
         6.41789678e-03]])